In [4]:
#Read the downloaded data.

import pandas as pd


#Consider data from 2010 to 2018 -- 2018 is the last year available 
year_range = range(10,19)
#Quarter 1 to 4
quarter_range = range(1,4+1)

#Take interview and diary data 
file_class = ['interview','diary']

file_category = {k: [] for k in file_class}
#List of files for interview - last character i stands for interview and d for diary
#1. FMLI - a file with characteristics, income, weights, and summary level expenditures for the CU.
#2. MEMI - a file with characteristics and income for each member in the CU.
#3. MTBI - a detailed monthly expenditure file categorized by Universal Classification Code (UCC).
#4. ITBI - a Consumer Unit monthly income file categorized by UCC.
#5. ITII - a Consumer Unit monthly imputed income file categorized by UCC
#6. NTAXI - a file with federal and state tax information for each tax unit in the CU
file_category['interview'] = ['fmli','memi','mtbi', 'itbi', 'itii', 'ntaxi']
file_category['diary'] = ['expd','fmld','memd','dtbd','dtid']

file_class_type = 'interview'
file_name = dict.fromkeys(file_category[file_class_type], "")
raw_data_folder = dict.fromkeys(file_category[file_class_type], "")
combined_csv = {}
file_list = {k: [] for k in file_category[file_class_type]}
file_type = 'fmli'


#Read only fmli for now
requested_file_type = 'fmli'
first_time = True
for year in year_range:
    #for file_type in file_types:
        folder_name = "C:/fodfile/intrvw" + str(year) + "/" + file_type + str(year)
        raw_data_folder[file_type] = folder_name

    #for file_type in file_types:
        for quarter in quarter_range:
            if quarter == 1: 
                quart5 = 'x'
            else:
                quart5 = ''
            file_name[file_type] = raw_data_folder[file_type] + str(quarter) + str(quart5) + ".csv"
            temp_df = pd.read_csv(file_name[file_type] ,dtype = 'str')
            print("Read " + file_name[file_type] )
            temp_df['YEAR'] = year
            temp_df['QUARTER'] = quarter
            if (first_time == True):
                combined_csv[requested_file_type] = temp_df
                first_time = False
            else:
                combined_csv[requested_file_type] = combined_csv[requested_file_type].append(temp_df, sort = 'False')
                
combined_csv[requested_file_type].info()

Read C:/fodfile/intrvw10/fmli101x.csv
Read C:/fodfile/intrvw10/fmli102.csv
Read C:/fodfile/intrvw10/fmli103.csv
Read C:/fodfile/intrvw10/fmli104.csv
Read C:/fodfile/intrvw11/fmli111x.csv
Read C:/fodfile/intrvw11/fmli112.csv
Read C:/fodfile/intrvw11/fmli113.csv
Read C:/fodfile/intrvw11/fmli114.csv
Read C:/fodfile/intrvw12/fmli121x.csv
Read C:/fodfile/intrvw12/fmli122.csv
Read C:/fodfile/intrvw12/fmli123.csv
Read C:/fodfile/intrvw12/fmli124.csv
Read C:/fodfile/intrvw13/fmli131x.csv
Read C:/fodfile/intrvw13/fmli132.csv
Read C:/fodfile/intrvw13/fmli133.csv
Read C:/fodfile/intrvw13/fmli134.csv
Read C:/fodfile/intrvw14/fmli141x.csv
Read C:/fodfile/intrvw14/fmli142.csv
Read C:/fodfile/intrvw14/fmli143.csv
Read C:/fodfile/intrvw14/fmli144.csv
Read C:/fodfile/intrvw15/fmli151x.csv
Read C:/fodfile/intrvw15/fmli152.csv
Read C:/fodfile/intrvw15/fmli153.csv
Read C:/fodfile/intrvw15/fmli154.csv
Read C:/fodfile/intrvw16/fmli161x.csv
Read C:/fodfile/intrvw16/fmli162.csv
Read C:/fodfile/intrvw16/fmli16

In [5]:
#read the description of variables
var_list = pd.read_csv('C:/fodfile/dictionary/variable_description.csv')
var_list.head()

,Survey,File,Variable Name,Variable description,Formula,Flag name,Section number,Section description,Section part,First year,First Quarter,Last quarter,Last year,Comment
0,INTERVIEW,FMLI,ACCESS,Access to living quarters.,NaN,ACCESS_,NaN,"CU characteristics, income, weights, and summa...",NaN,1980,1.0,4.0,1981.0,NaN
1,INTERVIEW,FMLI,ACCESS,Access to living quarters.,NaN,ACCESS_,NaN,"CU characteristics, income, weights, and summa...",NaN,1984,1.0,1.0,1998.0,NaN
2,INTERVIEW,OPB,ACQMETH,Method by which property was acquired,NaN,ACQMETH_,3,Owned Living Quarters and Other Owned Real Estate,B,1994,1.0,1.0,2006.0,NaN
3,INTERVIEW,OPB,ACQUIRMO,Month Property Acquired,NaN,ACQU_RMO,3,Owned Living Quarters and Other Owned Real Estate,B,1994,1.0,NaN,NaN,NaN
4,INTERVIEW,OPB,ACQUIRYR,Year Property Acquired,NaN,ACQU_RYR,3,Owned Living Quarters and Other Owned Real Estate,B,1994,1.0,NaN,NaN,NaN


In [10]:
#read the description of codes
code_list = pd.read_csv('C:/fodfile/dictionary/code_description.csv')
update_range =(code_list['File'] == requested_file_type.upper()) &  ( code_list['Last year'].isnull())
idx = code_list.groupby(['Survey', 'Variable '])['First year'].transform(max) == code_list['First year']
updated_code_list = code_list.loc[update_range,]
updated_code_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502 entries, 6319 to 6820
Data columns (total 10 columns):
Survey              502 non-null object
File                502 non-null object
Variable            502 non-null object
Code value          502 non-null object
Code description    502 non-null object
First year          502 non-null int64
First quarter       502 non-null int64
Last year           0 non-null float64
Last quarter        2 non-null float64
Comment             0 non-null object
dtypes: float64(2), int64(2), object(6)
memory usage: 43.1+ KB


In [11]:
#Only read the those variables that have existed for all the 10 years
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3


requested_range =(var_list['File'] == requested_file_type.upper()) & (var_list['First year'] < 2010) & ( var_list['Last year'].isnull())
modified_var_list = var_list.loc[requested_range,['Variable Name', 'Variable description' ,'Formula' ,'Flag name']].drop_duplicates(subset=['Variable Name'])
requested_fields = modified_var_list['Variable Name'].unique()

final_list_of_Columns = intersection(requested_fields, combined_csv[requested_file_type].columns)
print(len(modified_var_list['Variable Name'].unique()))


441


In [68]:
#Understand the variables of interest
select_list_of_Columns = ('YEAR','QUARTER','newid', 'cuid' ,  # CU id
            'region',   # region
            'state',    # state
            'cutenure', # housing tenure 
            'fam_size',  # family size
            'FAM_TYPE',  #Family Type is based on the relationship of members to reference person, whom physically occupy the address selected for the survey.
                          #"Own" children are sons and daughters children including step children and adopted
            'no_earnr', # number of earners
            'num_auto', # number of vehicles owned
            'popsize',  # population sizese(see codes)
            'MARITAL1',     #Marital status of reference person
            'sex_ref',  # sex reference person
            'age_ref',  # age reference person
            'age2',     # age of spouse
             'EARNCOMP', #Composition of earners
            'educ_ref', # education reference person (see codes)
            'EDUCA2', # education spouse (see codes)
            'ref_race', # race reference person (1=white, 2=black, 3= nat-am, 4=asian, 5=pac-isl, 6=multi-race)
            'race2',  # race spouse (1=white, 2=black, 3= nat-am, 4=asian, 5=pac-isl, 6=multi-race)
            'inc_hrs1', # hours per week by ref person
            'inc_hrs2', # hours per week by spouse
            'incweek1', # number weeks worked ref person
            'incweek2', # number weeks worked spouse
            'INCOMEY1',     #Employer from which reference person received the most earnings in past 12 months
            'INCOMEY2',     #Employer from which spouse received most earnings during the past 12 months
            'LUMPSUMX',     #Amount of lump sum payments from estates, trusts, royalties, alimony, prizes, or games of chance or from persons outside CU
            'SMSASTAT',     #Does CU reside inside a Metropolitan Statistical Area (MSA)?
            'MISCTAXX',     #Amount of other taxes paid which were not reported elsewhere during past 12 months
            'INCLASS2',
            'INCNONW1',     #Reason reference person did not work during the past 12 months
            'INCNONW2',     #Reason spouse did not work during the past 12 months
            'fincbtax',  # income before tax past 12 month
            'fincatax', # income after tax past 12 month
            'FINDRETX',     #Amount of money placed in a Self-employed retirement plan in past year for all CU members
            'FINLWT21',     #Calibration final weight for the full sample
            'fsalaryx', # wage and salary income before ded.
            'totexppq', # tot exp prev quarter
            'totexpcq', # tot exp curr quarter
            'majapppq', # major appliances prev quarter
            'majappcq', # major appliances curr quarter,
             'Foodpq', 'alcbevpq', 'foodcq', 'alcbevcq', #food and bevergaes
            'Houspq', 'houscq', #housing
            'HOUSEQCQ',     #House furnishings and equipment this quarter
            'HOUSEQPQ',     #House furnishings and equipment last quarter
            'HOUSOPCQ',     #Household operations this quarter
             'HOUSOPPQ',     #Household operations last quarter

            'Apparpq', 'apparcq', #apparel
            'Transpq', 'transcq',  #transport
            'Healthpq', 'healthcq', #medical care
            'Entertpq', 'entertcq', #Recreation        
            'Educapq', 'educacq', #Education and Communication 
            'Perinspq', 'perinscq', #Personal Insurance, Pension, Saving
            'GASMOCQ',     #Gasoline and motor oil this quarter
            'GASMOPQ',     #Gasoline and motor oil last quarter
            'Cashcopq', 'cashcocq' #Cash contribution
           )

select_list_of_Columns = [x.upper() for x in select_list_of_Columns]
print(select_list_of_Columns)

['YEAR', 'QUARTER', 'NEWID', 'CUID', 'REGION', 'STATE', 'CUTENURE', 'FAM_SIZE', 'FAM_TYPE', 'NO_EARNR', 'NUM_AUTO', 'POPSIZE', 'MARITAL1', 'SEX_REF', 'AGE_REF', 'AGE2', 'EARNCOMP', 'EDUC_REF', 'EDUCA2', 'REF_RACE', 'RACE2', 'INC_HRS1', 'INC_HRS2', 'INCWEEK1', 'INCWEEK2', 'INCOMEY1', 'INCOMEY2', 'LUMPSUMX', 'SMSASTAT', 'MISCTAXX', 'INCLASS2', 'INCNONW1', 'INCNONW2', 'FINCBTAX', 'FINCATAX', 'FINDRETX', 'FINLWT21', 'FSALARYX', 'TOTEXPPQ', 'TOTEXPCQ', 'MAJAPPPQ', 'MAJAPPCQ', 'FOODPQ', 'ALCBEVPQ', 'FOODCQ', 'ALCBEVCQ', 'HOUSPQ', 'HOUSCQ', 'HOUSEQCQ', 'HOUSEQPQ', 'HOUSOPCQ', 'HOUSOPPQ', 'APPARPQ', 'APPARCQ', 'TRANSPQ', 'TRANSCQ', 'HEALTHPQ', 'HEALTHCQ', 'ENTERTPQ', 'ENTERTCQ', 'EDUCAPQ', 'EDUCACQ', 'PERINSPQ', 'PERINSCQ', 'GASMOCQ', 'GASMOPQ', 'CASHCOPQ', 'CASHCOCQ']


In [13]:
#Replace codes in the variables of interest
lookup_and_replace_columns = [ 'REGION', 'STATE', 'CUTENURE', 'POPSIZE',  'EDUC_REF', 'REF_RACE' , 'RACE2', 'SMSASTAT',
                              'SEX_REF', 'EARNCOMP', 'EDUCA2','FAM_TYPE', 'INCOMEY1', 'INCOMEY2', 'MARITAL1', 'INCLASS2' ]
def lookup_and_replace(data_df, vlookup_ref):
    data_df[vlookup_ref] = data_df[vlookup_ref].fillna(-1) 
    data_df[vlookup_ref] = data_df[vlookup_ref].astype(int)
    data_df[vlookup_ref] = data_df[vlookup_ref].astype(str)
    
    vlookup_range =updated_code_list.loc[(updated_code_list['File'] == requested_file_type.upper()) & (updated_code_list['Variable '] == vlookup_ref),['Code value','Code description']]
    vlookup_dict = vlookup_range.set_index('Code value')['Code description'].to_dict()
    data_df[vlookup_ref] = data_df[vlookup_ref].map(vlookup_dict)
    

updated_df = combined_csv[requested_file_type].loc[:,select_list_of_Columns].copy()
updated_df['NEWID'] = updated_df['NEWID'].astype(str)
updated_df['YEAR'] = updated_df['YEAR'] + 2000


for columnname in lookup_and_replace_columns:
    lookup_and_replace(updated_df, columnname)
updated_df['SEX_REF'] = updated_df['SEX_REF'].replace( ['Reference person is female','Reference person is male'],['Female','Male'])
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
updated_df.head()

,YEAR,QUARTER,NEWID,CUID,REGION,STATE,CUTENURE,FAM_SIZE,FAM_TYPE,NO_EARNR,NUM_AUTO,POPSIZE,MARITAL1,SEX_REF,AGE_REF,AGE2,EARNCOMP,EDUC_REF,EDUCA2,REF_RACE,RACE2,INC_HRS1,INC_HRS2,INCWEEK1,INCWEEK2,INCOMEY1,INCOMEY2,LUMPSUMX,SMSASTAT,MISCTAXX,INCLASS2,INCNONW1,INCNONW2,FINCBTAX,FINCATAX,FINDRETX,FINLWT21,FSALARYX,TOTEXPPQ,TOTEXPCQ,MAJAPPPQ,MAJAPPCQ,FOODPQ,ALCBEVPQ,FOODCQ,ALCBEVCQ,HOUSPQ,HOUSCQ,HOUSEQCQ,HOUSEQPQ,HOUSOPCQ,HOUSOPPQ,APPARPQ,APPARCQ,TRANSPQ,TRANSCQ,HEALTHPQ,HEALTHCQ,ENTERTPQ,ENTERTCQ,EDUCAPQ,EDUCACQ,PERINSPQ,PERINSCQ,GASMOCQ,GASMOPQ,CASHCOPQ,CASHCOCQ
0,2010,1,1995545,199554,Midwest,Ohio,Owned w/mortgage:,2,Married Couple only,1,1,100-500 thousand,Married,Male,64,61,Spouse only,"High school (grades 9-12), no degree",Highschool graduate,White,White,.,20,0,52,NaN,"Private company, business or individual",.,Yes,.,.5000 - .6666,1,NaN,46184,47797,0,15395.99,13044,3601.9,5540.8001,0,0,338,0,676,0,1018,2314,58,16,80,40,111,0,536,270,694.7333,881.4667,187,112,0,0,310.1667,481.3334,84,42,387,781
1,2010,1,1995565,199556,South,Virginia,Owned w/mortgage:,2,Married Couple only,0,2,1-5 million,Married,Male,63,62,No Earners,NaN,"Masters, professional or doctorate degree",White,White,.,.,0,0,NaN,NaN,.,Yes,.,Incomplete reporting,1,1,0,0,0,12950.979,0,3427.1667,1528.5833,0,0,691.3333,0,345.6667,0,1685.8333,842.9167,0,0,50,100,750,0,300,340,0,0,0,0,0,0,0,0,150,300,0,0
2,2010,1,1995585,199558,Midwest,NaN,Owned w/o mortgage:,2,Married Couple only,2,0,Less than 100 thousand,Married,Male,57,44,Reverence Person and Spouse,Bachelors degree,"Some college, no degree",White,White,60,45,52,52,"Private company, business or individual","Private company, business or individual",.,No,3001,.8334 - 1.0000,NaN,NaN,162000,151479,9000,20667.144,160000,23648.7501,0,0,0,2775,40,0,0,5960,0,0,820,0,396,350,0,3142,0,2595,0,712,0,0,0,5294.7501,0,0,400,2400,0
3,2010,1,1995595,199559,West,Colorado,Owned w/mortgage:,3,Other families,2,1,1-5 million,Never married,Male,31,.,Reference Person and Others,"Some college, no degree",NaN,White,NaN,40,.,52,.,"Private company, business or individual",NaN,.,Yes,.,.6667 - .8333,NaN,NaN,81000,81000,0,19096.304,81000,9014.2501,0,0,0,1300,0,0,0,4599,0,0,0,0,120,0,0,900,0,156,0,120,0,0,0,1549.2501,0,0,900,0,0
4,2010,1,1995605,199560,South,Florida,Owned w/o mortgage:,2,Married Couple only,0,1,1-5 million,Married,Female,87,87,No Earners,"Masters, professional or doctorate degree","Masters, professional or doctorate degree",White,White,.,.,0,0,NaN,NaN,.,Yes,.,.1667 - .3333,1,1,20314,20314,0,16209.256,0,1589.8,3080.6,0,0,693.3333,0,1386.6667,0,359,701,0,0,38,27,0,70,220,440,194.8,409.6,96,0,0,0,0,0,440,220,0,20


In [14]:
#Understand if there are missing values that need to be imputed
pd.set_option('display.max_rows', None)
updated_df.isnull().sum()/updated_df.isnull().count()

YEAR       0.00
QUARTER    0.00
NEWID      0.00
CUID       0.00
REGION     0.02
STATE      0.12
CUTENURE   0.00
FAM_SIZE   0.00
FAM_TYPE   0.00
NO_EARNR   0.00
NUM_AUTO   0.00
POPSIZE    0.00
MARITAL1   0.00
SEX_REF    0.00
AGE_REF    0.00
AGE2       0.10
EARNCOMP   0.00
EDUC_REF   0.01
EDUCA2     0.52
REF_RACE   0.00
RACE2      0.51
INC_HRS1   0.07
INC_HRS2   0.14
INCWEEK1   0.00
INCWEEK2   0.10
INCOMEY1   0.34
INCOMEY2   0.66
LUMPSUMX   0.20
SMSASTAT   0.00
MISCTAXX   0.21
INCLASS2   0.00
INCNONW1   0.66
INCNONW2   0.85
FINCBTAX   0.00
FINCATAX   0.42
FINDRETX   0.00
FINLWT21   0.00
FSALARYX   0.00
TOTEXPPQ   0.00
TOTEXPCQ   0.00
MAJAPPPQ   0.00
MAJAPPCQ   0.00
FOODPQ     0.00
ALCBEVPQ   0.00
FOODCQ     0.00
ALCBEVCQ   0.00
HOUSPQ     0.00
HOUSCQ     0.00
HOUSEQCQ   0.00
HOUSEQPQ   0.00
HOUSOPCQ   0.00
HOUSOPPQ   0.00
APPARPQ    0.00
APPARCQ    0.00
TRANSPQ    0.00
TRANSCQ    0.00
HEALTHPQ   0.00
HEALTHCQ   0.00
ENTERTPQ   0.00
ENTERTCQ   0.00
EDUCAPQ    0.00
EDUCACQ    0.00
PERINSPQ

In [16]:
#Fam size vs type to undertand what consumer units to target
count_table = pd.crosstab(updated_df["FAM_SIZE"], updated_df["FAM_TYPE"])
count_table

FAM_TYPE,All other Husband and wife families,Married Couple only,"Married Couple, own children only oldest child >= 6, <= 17","Married Couple, own children only, oldest child < 6","Married Couple, own children only, oldest child > 17","One parent, female, own children, at least one age < 18","One parent, male, own children at least one age < 18",Other families,Single consumers
FAM_SIZE,,,,,,,,,
1,0,0,0,0,0,0,0,0,69792
10,63,0,17,0,22,0,0,20,0
11,42,0,15,0,10,1,0,22,0
12,20,0,1,0,24,0,0,12,0
13,7,0,5,0,12,0,0,7,0
14,8,0,0,0,5,0,0,9,0
15,1,0,4,0,0,0,0,1,0
16,2,0,0,0,0,0,0,0,0
2,0,49523,0,0,0,4054,1137,19674,0


In [17]:
#Select single consumers and those with incomplete reporting
single_consumer_condition = (updated_df['FAM_TYPE']=='Single consumers')&(updated_df['INCLASS2'] !='Incomplete reporting')
single_consumer_df = updated_df.loc[single_consumer_condition,].copy()
single_consumer_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58375 entries, 7 to 5568
Data columns (total 68 columns):
YEAR        58375 non-null int64
QUARTER     58375 non-null int64
NEWID       58375 non-null object
CUID        58375 non-null object
REGION      57089 non-null object
STATE       50716 non-null object
CUTENURE    58375 non-null object
FAM_SIZE    58375 non-null object
FAM_TYPE    58375 non-null object
NO_EARNR    58375 non-null object
NUM_AUTO    58375 non-null object
POPSIZE     58152 non-null object
MARITAL1    58375 non-null object
SEX_REF     58375 non-null object
AGE_REF     58375 non-null object
AGE2        45936 non-null object
EARNCOMP    58375 non-null object
EDUC_REF    57789 non-null object
EDUCA2      0 non-null object
REF_RACE    58375 non-null object
RACE2       0 non-null object
INC_HRS1    53030 non-null object
INC_HRS2    45936 non-null object
INCWEEK1    58375 non-null object
INCWEEK2    45936 non-null object
INCOMEY1    34612 non-null object
INCOMEY2    0 non-

In [19]:
#Bin the age data for segmentation
age_bins = [20, 25, 30, 35, 40, 50, 60, 70, 80, 100]
single_consumer_df['AGE_BIN']= pd.cut(single_consumer_df['AGE_REF'].astype(int), age_bins )
#print(single_consumer_df['AGE_BIN'])
single_consumer_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 58375 entries, 7 to 5568
Data columns (total 69 columns):
YEAR        58375 non-null int64
QUARTER     58375 non-null int64
NEWID       58375 non-null object
CUID        58375 non-null object
REGION      57089 non-null object
STATE       50716 non-null object
CUTENURE    58375 non-null object
FAM_SIZE    58375 non-null object
FAM_TYPE    58375 non-null object
NO_EARNR    58375 non-null object
NUM_AUTO    58375 non-null object
POPSIZE     58152 non-null object
MARITAL1    58375 non-null object
SEX_REF     58375 non-null object
AGE_REF     58375 non-null object
AGE2        45936 non-null object
EARNCOMP    58375 non-null object
EDUC_REF    57789 non-null object
EDUCA2      0 non-null object
REF_RACE    58375 non-null object
RACE2       0 non-null object
INC_HRS1    53030 non-null object
INC_HRS2    45936 non-null object
INCWEEK1    58375 non-null object
INCWEEK2    45936 non-null object
INCOMEY1    34612 non-null object
INCOMEY2    0 non-

In [69]:
#Pivot data for segmentation
grouped = single_consumer_df.groupby(['REF_RACE','MARITAL1','INCLASS2','SEX_REF', 'AGE_REF' ])['NEWID' ].count()
pivoted = pd.pivot_table(single_consumer_df, values='NEWID', index=['INCLASS2','MARITAL1'],
                    columns=['REF_RACE','SEX_REF', 'AGE_BIN'], aggfunc='count')
print(pivoted)

#pivoted.to_excel("CES_singleconsumer_pivot.xlsx")

REF_RACE                         Asian                                      \
SEX_REF                         Female                                       
AGE_BIN                       (20, 25] (25, 30] (30, 35] (35, 40] (40, 50]   
INCLASS2        MARITAL1                                                     
.1667 - .3333   Divorced           nan      nan     1.00     1.00     5.00   
                Married            nan     1.00      nan      nan     3.00   
                Never married    30.00    14.00     5.00      nan     6.00   
                Separated          nan      nan      nan     1.00      nan   
                Widowed            nan      nan      nan      nan     1.00   
...                                ...      ...      ...      ...      ...   
Less than .1667 Divorced           nan      nan     1.00      nan     6.00   
                Married            nan      nan      nan     1.00      nan   
                Never married    68.00    16.00     7.00     1.0

In [64]:
#create new dataframe to store segmentation results  
from scipy import stats
import math
import itertools

single_consumer_df ['Perc_Spend_Food']= single_consumer_df['FOODCQ'].astype('float')/single_consumer_df['TOTEXPCQ'].astype('float')
single_consumer_df ['Perc_Spend_Housing']= single_consumer_df['HOUSCQ'].astype('float')/single_consumer_df['TOTEXPCQ'].astype('float')
single_consumer_df ['Perc_Spend_Apparel']= single_consumer_df['APPARCQ'].astype('float')/single_consumer_df['TOTEXPCQ'].astype('float')
single_consumer_df ['Perc_Spend_Transport']= single_consumer_df['TRANSCQ'].astype('float')/single_consumer_df['TOTEXPCQ'].astype('float')
single_consumer_df ['Perc_Spend_Education']= single_consumer_df['EDUCACQ'].astype('float')/single_consumer_df['TOTEXPCQ'].astype('float')
single_consumer_df ['Perc_Spend_Entertainment']= single_consumer_df['ENTERTCQ'].astype('float')/single_consumer_df['TOTEXPCQ'].astype('float')
single_consumer_df ['Perc_Spend_InsuranceandSavings']= single_consumer_df['PERINSCQ'].astype('float')/single_consumer_df['TOTEXPCQ'].astype('float')

spend_Catgories_originaldf = ['Perc_Spend_Food', 'Perc_Spend_Housing', 'Perc_Spend_Apparel',\
                              'Perc_Spend_Transport', 'Perc_Spend_Education', 'Perc_Spend_Entertainment',\
                              'Perc_Spend_InsuranceandSavings']
#spend_Catgories = ['Food', 'Housing', 'Apparel', 'Transport', 'Education', 'Entertainment', 'InsuranceandSavings']
races = ['Asian', 'White', 'Black']
income_bins = ['Less than .1667', '.1667 - .3333', '.3334 - .4999', '.5000 - .6666', '.6667 - .8333', '.8334 - 1.0000' ]
age_brackets =   pd.IntervalIndex.from_breaks(age_bins)

Reasons_for_being_single =  ['Divorced','Married','Never married','Separated','Widowed']

#testing_df = pd.DataFrame(columns=['Race','Income_Percentile','Age_Bracket', 'Marital_Status'])
                                

skewness_index = 4
kurtosis_index = 5


combined_list = [races, income_bins, age_brackets, Reasons_for_being_single,spend_Catgories_originaldf ]
segments = list(itertools.product(*combined_list))

idx = ['SC_{}'.format(i) for i in range(1, len(segments)+1)]

testing_df = pd.DataFrame(segments, index=idx, columns=['Race','Income_Percentile','Age_Bracket', 'Marital_Status', 'Spend_Categories'])
    
    
print(   testing_df )

          Race Income_Percentile Age_Bracket Marital_Status  \
SC_1     Asian   Less than .1667    (20, 25]       Divorced   
SC_2     Asian   Less than .1667    (20, 25]       Divorced   
SC_3     Asian   Less than .1667    (20, 25]       Divorced   
SC_4     Asian   Less than .1667    (20, 25]       Divorced   
SC_5     Asian   Less than .1667    (20, 25]       Divorced   
SC_6     Asian   Less than .1667    (20, 25]       Divorced   
SC_7     Asian   Less than .1667    (20, 25]       Divorced   
SC_8     Asian   Less than .1667    (20, 25]        Married   
SC_9     Asian   Less than .1667    (20, 25]        Married   
SC_10    Asian   Less than .1667    (20, 25]        Married   
SC_11    Asian   Less than .1667    (20, 25]        Married   
SC_12    Asian   Less than .1667    (20, 25]        Married   
SC_13    Asian   Less than .1667    (20, 25]        Married   
SC_14    Asian   Less than .1667    (20, 25]        Married   
SC_15    Asian   Less than .1667    (20, 25]  Never mar

In [46]:
#95 confidence level
def get_95_ci(array_1, array_2):
    sample_1_n = array_1.shape[0]
    sample_2_n = array_2.shape[0]
    sample_1_mean = array_1.mean()
    sample_2_mean = array_2.mean()
    sample_1_var = array_1.var()
    sample_2_var = array_2.var()
    mean_difference = sample_2_mean - sample_1_mean
    std_err_difference = math.sqrt((sample_1_var/sample_1_n)+(sample_2_var/sample_2_n))
    margin_of_error = 1.96 * std_err_difference
    ci_lower = mean_difference - margin_of_error
    ci_upper = mean_difference + margin_of_error
    return ci_lower, ci_upper


#get_95_ci(single_consumer_df.loc[male_data,'Perc_Spend_Food'].dropna(), single_consumer_df.loc[female_data,'Perc_Spend_Food'].dropna())

In [65]:
#fill segment dataframe with test results
for race in races:
    for income_precentile in income_bins:
        for age_bracket in age_brackets:
            for marital_status in Reasons_for_being_single:
                for category in spend_Catgories_originaldf:
                    segment_data = (testing_df['Race']==race) & (testing_df['Age_Bracket']==age_bracket)\
                    & (testing_df['Income_Percentile']==income_precentile) & (testing_df['Marital_Status']==marital_status)\
                    & (testing_df['Spend_Categories']==category)

                    female_data = (single_consumer_df['REF_RACE']==race) & (single_consumer_df['AGE_BIN']==age_bracket)\
                    & (single_consumer_df['INCLASS2']==income_precentile) & (single_consumer_df['MARITAL1']==marital_status)\
                    & (single_consumer_df['SEX_REF']=='Female')

                    male_data = (single_consumer_df['REF_RACE']==race) & (single_consumer_df['AGE_BIN']==age_bracket)\
                    & (single_consumer_df['INCLASS2']==income_precentile) & (single_consumer_df['MARITAL1']==marital_status)\
                    & (single_consumer_df['SEX_REF']=='Male')

                    temp_data_for_analysis_1 = single_consumer_df.loc[male_data,category].dropna()
                    temp_data_for_analysis_2 = single_consumer_df.loc[female_data,category].dropna()
                    
                    if(temp_data_for_analysis_1.empty==False):           
                        testing_df.loc[segment_data,'samplesize_male'] = temp_data_for_analysis_1.size
                        testing_df.loc[segment_data,'mean_male'] = temp_data_for_analysis_1.mean()
                        testing_df.loc[segment_data,'skewness_male'] = stats.describe(temp_data_for_analysis_1)[skewness_index]
                        testing_df.loc[segment_data,'kurtosis_male'] = stats.describe(temp_data_for_analysis_1)[kurtosis_index]
                    if(temp_data_for_analysis_2.empty==False):           
                        testing_df.loc[segment_data,'samplesize_female'] = temp_data_for_analysis_2.size
                        testing_df.loc[segment_data,'mean_female'] = temp_data_for_analysis_2.mean()
                        testing_df.loc[segment_data,'skewness_female'] = stats.describe(temp_data_for_analysis_2)[skewness_index]
                        testing_df.loc[segment_data,'kurtosis_female'] = stats.describe(temp_data_for_analysis_2)[kurtosis_index]


                    if((temp_data_for_analysis_1.empty==False)&(temp_data_for_analysis_2.empty==False)): 
                        statistic, pvalue = stats.ttest_ind(temp_data_for_analysis_1, temp_data_for_analysis_2)
                        ci95_lower, ci95_upper =  get_95_ci(temp_data_for_analysis_1, temp_data_for_analysis_2)
                        testing_df.loc[segment_data,'p_value'] = pvalue
                        testing_df.loc[segment_data,'ci_95_lower'] = ci95_lower
                        testing_df.loc[segment_data,'ci_95_upper'] = ci95_upper

pd.set_option('display.max_rows', 10)   
print(testing_df)

          Race Income_Percentile Age_Bracket Marital_Status  \
SC_1     Asian   Less than .1667    (20, 25]       Divorced   
SC_2     Asian   Less than .1667    (20, 25]       Divorced   
SC_3     Asian   Less than .1667    (20, 25]       Divorced   
SC_4     Asian   Less than .1667    (20, 25]       Divorced   
SC_5     Asian   Less than .1667    (20, 25]       Divorced   
...        ...               ...         ...            ...   
SC_5666  Black    .8334 - 1.0000   (80, 100]        Widowed   
SC_5667  Black    .8334 - 1.0000   (80, 100]        Widowed   
SC_5668  Black    .8334 - 1.0000   (80, 100]        Widowed   
SC_5669  Black    .8334 - 1.0000   (80, 100]        Widowed   
SC_5670  Black    .8334 - 1.0000   (80, 100]        Widowed   

                       Spend_Categories  samplesize_male  mean_male  \
SC_1                    Perc_Spend_Food              nan        nan   
SC_2                 Perc_Spend_Housing              nan        nan   
SC_3                 Perc_Spen